<a href="https://colab.research.google.com/github/stephanefschwarz/name-entity-recognition/blob/master/name-entity-recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [40]:
!pip install Spacy
import spacy
import tqdm

!python -m spacy download pt

✔ Download and installation successful
You can now load the model via spacy.load('pt_core_news_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/pt_core_news_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/pt
You can now load the model via spacy.load('pt')


In [0]:
from spacy import displacy
import random
from spacy.util import minibatch, compounding

In [0]:
data = [('Almira é uma ótima pessoa, gosto muito dela', {'entities':[(0,6,'PER')]}),
        ('João foi para Bahia nas férias', {'entities':[(14,19,'LOC'), (0,4,'PER')]}),
        ('Fulano foi visitar João na casa de praia', {'entities':[(19,23,'LOC')]}),
        ('No meio do ano irei para São Paulo fazer mais um curso', {'entities':[(25,34,'LOC')]}),
        ('O sonho dela era ir para Austrália visitar seu irmão', {'entities':[(25,34,'LOC')]}),
        ('Em 15/07/1988 nasceu essa linda criança', {'entities':[(3,13,'DATE')]}),
        ('Data de Prisão: 10/01/2018', {'entities':[(16,26,'DATE')]}),
        ('No dia 01/02/2016 foi decretado a sentença', {'entities':[(7,17,'DATE')]}),
        ('A data da festa foi 07/05/2018', {'entities':[(20,30,'DATE')]}),
        ('Em 15/07/1988 nasceu essa linda criança', {'entities': [(3, 13, 'DATE')]}), 
        ('Data de prisão: 10/01/2018', {'entities': [(16, 26, 'DATE')]}),
        ('No dia 01/02/2016 foi decretada a setença', {'entities': [(7, 17, 'DATE')]}),
        ('A data da festa foi 07/05/2019', {'entities': [(20, 30, 'DATE')]}),
        ('João, portador do cpf 123.456.789-65, foi encontrado', {'entities': [(0, 4, 'PER'), (18, 32, 'CPF')]}),
        ('O cpf 064.468.404-62 pertence a Maria', {'entities': [(6, 20, 'CPF'), (32, 37, 'PER')]})
        ]

TRAIN_DATA = data

In [43]:
nlp = spacy.blank('pt')

tex = nlp('Maria está se mudando para Paris. No dia 01/02/2019 ela irá partir.')
displacy.render(tex, style='ent', jupyter=True)

/usr/lib/python3.6/runpy.py:193: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  "__main__", mod_spec)


In [0]:
ner = nlp.create_pipe('ner')
nlp.add_pipe(ner, last=True)

In [0]:
for _, ano in data:
  for ent in ano.get('entities'):
    ner.add_label(ent[2])

In [0]:
other = [pipe for pipe in nlp.pipe_names if pipe !='ner']
with nlp.disable_pipes(*other):
  opt = nlp.begin_training()
  for i in range(30):
    random.shuffle(data)
    loss = {}

    for text, annot in data:

      nlp.update([text],
                 [annot],
                 drop=0.5,
                 sgd=opt,
                 losses=loss)

In [47]:
for text, _ in data:
  doc = nlp(text)
  displacy.render(doc, style='ent', jupyter=True)

In [48]:
text = nlp('Maria está se mudando para Paris. No dia 01/02/2019 ela irá partir.')
displacy.render(text, style='ent', jupyter=True)

In [0]:
nlp.to_disk('/tmp/model.md')

**RETRAIN THE MODEL**

In [0]:
new = [('XXXXXX é uma ótima pessoa, gosto muito dela', {'entities':[(0,7,'NEW')]}),
        ('Foi visitar João na XXXXXX e praia', {'entities':[(20,27,'NEW')]}),
        ('No meio do ano irei para XXXXXX fazer mais um curso', {'entities':[(25,32,'NEW')]}),
        ('O sonho dela era ir para XXXXXX visitar seu irmão', {'entities':[(25,32,'NEW')]})
        ]

In [0]:
nlp2 = spacy.load('/tmp/model.md')

ner = nlp2.get_pipe('ner')

In [0]:
for _, ano in new:
  for ent in ano.get('entities'):
    ner.add_label(ent[2])

In [0]:
other = [pipe for pipe in nlp2.pipe_names if pipe !='ner']
with nlp2.disable_pipes(*other):
  opt = nlp2.begin_training()
  for i in range(30):
    random.shuffle(new)
    loss = {}

    for text, annot in new:

      nlp2.update([text],
                 [annot],
                 drop=0.5,
                 sgd=opt,
                 losses=loss)

In [61]:
for text, _ in new:
  doc = nlp2(text)
  displacy.render(doc, style='ent', jupyter=True)

/usr/lib/python3.6/runpy.py:193: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  "__main__", mod_spec)


/usr/lib/python3.6/runpy.py:193: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  "__main__", mod_spec)


In [62]:
text = nlp2('Maria está se mudando para Paris. No dia 01/02/2019 ela irá partir.')
displacy.render(text, style='ent', jupyter=True)